# Day 19: Aplenty

In [ ]:
import kotlin.io.path.Path
import kotlin.io.path.readLines

val data = Path("input.txt").readLines()

In [ ]:
data class Part(val x: Int, val m: Int, val a: Int, val s: Int) {
    companion object {
        fun fromLine(line: String): Part {
            val (x, m, a, s) = line.drop(1).dropLast(1)
                .split(',')
                .map { it.drop(2).toInt() }
            return Part(x, m, a, s)
        }
    }
}

data class Rule(val category: Char, val ltGt: Char, val value: Int, val target: String) {
    fun matches(part: Part): Boolean {
        val partValue = when (category) {
            'x' -> part.x
            'm' -> part.m
            'a' -> part.a
            's' -> part.s
            else -> throw IllegalArgumentException("AAAAAAHHHHH")
        }
        return if (ltGt == '>') partValue > value else partValue < value
    }

    companion object {
        fun fromRuleString(ruleString: String): Rule {
            val category = ruleString[0]
            val ltGt = ruleString[1]
            val (value, target) = ruleString.drop(2).split(':')
            
            return Rule(category, ltGt, value.toInt(), target)
        }
    }
}

class Workflow(rulesString: String) {
    val rules: List<Rule>
    val end: String

    init {
        val ruleStrings = rulesString.split(',')
        end = ruleStrings.last()
        rules = ruleStrings.dropLast(1)
            .map { Rule.fromRuleString(it) }
    }

    fun apply(part: Part): String {
        for (rule in rules) {
            if (rule.matches(part)) {
                return rule.target
            }
        }
        return end
    }
}

## Part 1

In [ ]:
val emptyIndex = data.indexOfFirst { it.isEmpty() }
val workflows = data.take(emptyIndex)
    .map {
        val (name, rules) = it.dropLast(1).split('{')
        name to Workflow(rules)
    }
    .toMap()
val parts = data.drop(emptyIndex + 1)
    .map { Part.fromLine(it) }

In [ ]:
val acceptedParts = mutableListOf<Part>()

for (part in parts) {
    var target = "in"
    while (target != "R" && target != "A") {
        target = workflows[target]!!.apply(part)
    }
    if (target == "A") {
        acceptedParts.add(part)
    }
}

acceptedParts.sumOf { (x, m, a, s) -> x + m + a + s }

## Part 2

In [ ]:
data class Range(
    val sx: Long, val ex: Long,
    val sm: Long, val em: Long,
    val sa: Long, val ea: Long,
    val ss: Long, val es: Long
)

In [ ]:
fun getValueRanges(s: Long, e: Long, value: Long, op: Char): Pair<Pair<Long, Long>?, Pair<Long, Long>?> {
    return if (op == '<') {
        val first = if (s >= value) null else s to min(e, value - 1)
        val second = if (e < value) null else max(value, s) to e
        first to second
    } else { // >
        val first = if (e <= value) null else max(s, value + 1) to e
        val second = if (s > value) null else s to min(value, e)
        first to second
    }
}

In [ ]:
fun search(target: String, range: Range): Long {
    val workflow = workflows[target]!!
    var trackedRange = range
    var result = 0L
    for (rule in workflow.rules) {
        val (matchRange, otherRange) = when (rule.category) {
            'x' -> getValueRanges(trackedRange.sx, trackedRange.ex, rule.value.toLong(), rule.ltGt)
            'm' -> getValueRanges(trackedRange.sm, trackedRange.em, rule.value.toLong(), rule.ltGt)
            'a' -> getValueRanges(trackedRange.sa, trackedRange.ea, rule.value.toLong(), rule.ltGt)
            's' -> getValueRanges(trackedRange.ss, trackedRange.es, rule.value.toLong(), rule.ltGt)
            else -> throw IllegalArgumentException("NOOOOOOOOOOOOOOOOOO")
        }
        if (matchRange != null) {
            val nextRange = Range(
                if (rule.category == 'x') matchRange.first else trackedRange.sx,
                if (rule.category == 'x') matchRange.second else trackedRange.ex,
                if (rule.category == 'm') matchRange.first else trackedRange.sm,
                if (rule.category == 'm') matchRange.second else trackedRange.em,
                if (rule.category == 'a') matchRange.first else trackedRange.sa,
                if (rule.category == 'a') matchRange.second else trackedRange.ea,
                if (rule.category == 's') matchRange.first else trackedRange.ss,
                if (rule.category == 's') matchRange.second else trackedRange.es
            )
            result += when (rule.target) {
                "A" -> (nextRange.ex - nextRange.sx + 1) *
                        (nextRange.em - nextRange.sm + 1) *
                        (nextRange.ea - nextRange.sa + 1) *
                        (nextRange.es - nextRange.ss + 1)
                "R" -> 0
                else -> search(rule.target, nextRange)
            }
        }
        if (otherRange == null) {
            return result
        } else {
            trackedRange = Range(
                if (rule.category == 'x') otherRange.first else trackedRange.sx,
                if (rule.category == 'x') otherRange.second else trackedRange.ex,
                if (rule.category == 'm') otherRange.first else trackedRange.sm,
                if (rule.category == 'm') otherRange.second else trackedRange.em,
                if (rule.category == 'a') otherRange.first else trackedRange.sa,
                if (rule.category == 'a') otherRange.second else trackedRange.ea,
                if (rule.category == 's') otherRange.first else trackedRange.ss,
                if (rule.category == 's') otherRange.second else trackedRange.es
            )
        }
    }
    result += when (workflow.end) {
        "A" -> (trackedRange.ex - trackedRange.sx + 1) *
                (trackedRange.em - trackedRange.sm + 1) *
                (trackedRange.ea - trackedRange.sa + 1) *
                (trackedRange.es - trackedRange.ss + 1)

        "R" -> 0
        else -> search(workflow.end, trackedRange)
    }
    return result
}

In [ ]:
search("in", Range(1, 4000, 1, 4000, 1, 4000, 1, 4000))